In [1]:
import sqlite3
import pandas as pd

DATA_FILE_PATH = "../data/clean_data.csv"

In [3]:
conn = sqlite3.connect("usa_pollution.db")
df = pd.read_csv(DATA_FILE_PATH)

In [5]:
cursor = conn.cursor()

location_columns = ["State", "County", "City", "Latitude", "Longitude"]
locations_df = df[location_columns].drop_duplicates().reset_index(drop=True)

locations_df.to_sql("locations", conn, if_exists="append", index=False)

location_query = "SELECT id, state, county, city, latitude, longitude FROM locations"
location_df = pd.read_sql(location_query, conn)

location_df.columns = ['id',"State", "County", "City", "Latitude", "Longitude"]

df = df.merge(location_df, on=["State", "County", "City", "Latitude", "Longitude"], how="left")
df.rename(columns={"id": "location"}, inplace=True)

pollutant_columns = [
    "O3 Mean", "O3 1st Max Value", "O3 1st Max Hour", "O3 AQI",
    "CO Mean", "CO 1st Max Value", "CO 1st Max Hour", "CO AQI",
    "SO2 Mean", "SO2 1st Max Value", "SO2 1st Max Hour", "SO2 AQI",
    "NO2 Mean", "NO2 1st Max Value", "NO2 1st Max Hour", "NO2 AQI",
    "Year", "Month", "Day", "location"
]

pollutants_df = df[pollutant_columns]

pollutants_df.columns = [
    "o3_mean", "o3_max_value", "o3_first_max_hour", "o3_aqi",
    "co_mean", "co_max_value", "co_first_max_hour", "co_aqi",
    "so2_mean", "so2_max_value", "so2_first_max_hour", "so2_aqi",
    "no2_mean", "no2_max_value", "no2_first_max_hour", "no2_aqi",
    "year", "month", "day", "location"
]

pollutants_df.to_sql("pollutants", conn, if_exists="append", index=False)

conn.close()

      id       State        County           City   Latitude   Longitude
0      1     Arizona      Maricopa        Phoenix  33.448437 -112.074141
1      2     Arizona      Maricopa     Scottsdale  33.494219 -111.926018
2      3     Arizona          Pima         Tucson  32.222876 -110.974847
3      4  California  Contra Costa        Concord  37.976852 -122.033562
4      5  California  Contra Costa  Bethel Island  38.029034 -121.640009
..   ...         ...           ...            ...        ...         ...
147  148      Nevada         Clark  Sunrise Manor  36.197298 -115.045292
148  149        Utah     Salt Lake        Midvale  40.614917 -111.910661
149  150     Florida       Broward          Davie  26.062866  -80.233104
150  151   Tennessee        Shelby        Memphis  35.146025  -90.051764
151  152     Wyoming       Laramie       Cheyenne  41.139981 -104.820246

[152 rows x 6 columns]
